In [7]:
import os
import random
import re
from PIL import Image
from libtiff import TIFF
import numpy as np
from shutil import copyfile

DATA_PATH = '/home/yifanc3/dataset/data/'
FRAME_PATH = DATA_PATH+'frames_128/'
MASK_PATH = DATA_PATH+'masks_128/'
ANNO_PATH = DATA_PATH+'annotations_128/'
SAVE_PATH = '/home/yifanc3/dataset/data/selected_128/'


#annotated_mask
# this is wrong , we need to split annotation image to 64*64 too.
# annotated_mask = np.load(os.path.join(DATA_PATH,'annotated_mask.tif')) # only 0 has annotated 
                         
# Create folders to hold features and nonfeatures

folders = ['frame_0', 'mask_0', 'frame_1', 'mask_1']

# os.makedirs(SAVE_PATH)
# for folder in folders:
#     os.makedirs(SAVE_PATH + folder)

  
# Get all frames and masks, sort them, shuffle them to generate data sets.

all_frames = os.listdir(FRAME_PATH)
all_masks = os.listdir(MASK_PATH)

#read all the masks to see if 1/0 >= 10/(64*64)
# the ratio need to be calculated by histgram (plot histogram of how large ratio is the label/nolabel )
                         
Ratios = []
num_files = 0
num_anno = 0
num_feature_file = 0
#select files with feature
for mask in all_masks:
    num_files += 1
    # load annotations
    filename = os.path.join(ANNO_PATH, mask)
    tif = TIFF.open(filename)
    img = tif.read_image().astype(int) # 0 means annotated
    if 0 not in np.unique(img):
        continue
    print(mask)
    num_anno += 1
    #load mask image
    filename = os.path.join(MASK_PATH, mask)
    tif = TIFF.open(filename)
    mask_arr = tif.read_image().astype(int)
    mask_arr = np.where(mask_arr==2.0, 0.0, mask_arr)
    mask_arr = 1 - mask_arr # so 1 is the mask
    # calculate ratio
    no_1 = np.count_nonzero(mask_arr)
    ratio = no_1/(128*128)
    if(no_1 != 0):
        Ratios.append(ratio)
    if(no_1 >= 20):
        num_feature_file += 1
        # load frames
        filename = os.path.join(FRAME_PATH, mask)
        tif = TIFF.open(filename)
        frame_arr = tif.read_image()
        # save to file masks_1 and load frames
        np.save(os.path.join(SAVE_PATH+'frame_1/',mask[0:-4]+'.npy'), frame_arr)
        np.save(os.path.join(SAVE_PATH+'mask_1/',mask[0:-4]+'.npy'),mask_arr)
    

    

001_020.tif
001_021.tif
001_022.tif
001_023.tif
001_024.tif
001_025.tif
001_026.tif
001_027.tif
002_018.tif
002_019.tif
002_020.tif
002_021.tif
002_022.tif
002_023.tif
002_024.tif
002_025.tif
002_026.tif
002_027.tif
002_028.tif
003_015.tif
003_016.tif
003_017.tif
003_018.tif
003_019.tif
003_020.tif
003_021.tif
003_022.tif
003_023.tif
003_024.tif
003_025.tif
003_026.tif
003_027.tif
003_028.tif
003_029.tif
004_014.tif
004_015.tif
004_016.tif
004_017.tif
004_018.tif
004_019.tif
004_020.tif
004_021.tif
004_022.tif
004_023.tif
004_024.tif
004_025.tif
004_026.tif
004_027.tif
004_028.tif
004_029.tif
004_030.tif
005_013.tif
005_014.tif
005_015.tif
005_016.tif
005_017.tif
005_018.tif
005_019.tif
005_020.tif
005_021.tif
005_022.tif
005_023.tif
005_024.tif
005_025.tif
005_026.tif
005_027.tif
005_028.tif
005_029.tif
005_030.tif
005_031.tif
005_032.tif
005_033.tif
005_034.tif
005_035.tif
006_013.tif
006_014.tif
006_015.tif
006_016.tif
006_017.tif
006_018.tif
006_019.tif
006_020.tif
006_021.tif
006_

024_007.tif
024_008.tif
024_009.tif
024_010.tif
024_011.tif
024_012.tif
024_013.tif
024_014.tif
024_015.tif
024_016.tif
024_017.tif
024_018.tif
024_019.tif
024_020.tif
024_021.tif
024_022.tif
024_023.tif
024_024.tif
024_025.tif
024_026.tif
024_027.tif
024_028.tif
024_029.tif
024_030.tif
024_031.tif
024_032.tif
024_033.tif
024_034.tif
024_035.tif
024_036.tif
024_037.tif
024_038.tif
024_039.tif
024_040.tif
024_041.tif
024_042.tif
024_043.tif
024_044.tif
024_045.tif
024_046.tif
024_047.tif
024_048.tif
025_008.tif
025_009.tif
025_010.tif
025_011.tif
025_012.tif
025_013.tif
025_014.tif
025_015.tif
025_016.tif
025_017.tif
025_018.tif
025_019.tif
025_020.tif
025_021.tif
025_022.tif
025_023.tif
025_024.tif
025_025.tif
025_026.tif
025_027.tif
025_028.tif
025_029.tif
025_030.tif
025_031.tif
025_032.tif
025_033.tif
025_034.tif
025_035.tif
025_036.tif
025_037.tif
025_038.tif
025_039.tif
025_040.tif
025_041.tif
025_042.tif
025_043.tif
025_044.tif
025_045.tif
025_046.tif
025_047.tif
025_048.tif
026_

039_049.tif
039_050.tif
039_051.tif
039_052.tif
039_053.tif
039_054.tif
039_055.tif
040_009.tif
040_010.tif
040_011.tif
040_012.tif
040_013.tif
040_014.tif
040_015.tif
040_016.tif
040_017.tif
040_018.tif
040_019.tif
040_020.tif
040_021.tif
040_022.tif
040_023.tif
040_024.tif
040_025.tif
040_026.tif
040_027.tif
040_028.tif
040_029.tif
040_030.tif
040_031.tif
040_032.tif
040_033.tif
040_034.tif
040_035.tif
040_036.tif
040_037.tif
040_038.tif
040_039.tif
040_040.tif
040_041.tif
040_042.tif
040_043.tif
040_044.tif
040_045.tif
040_046.tif
040_047.tif
040_048.tif
040_049.tif
040_050.tif
040_051.tif
040_052.tif
040_053.tif
040_054.tif
040_055.tif
040_056.tif
041_008.tif
041_009.tif
041_010.tif
041_011.tif
041_012.tif
041_013.tif
041_014.tif
041_015.tif
041_016.tif
041_017.tif
041_018.tif
041_019.tif
041_020.tif
041_021.tif
041_022.tif
041_023.tif
041_024.tif
041_025.tif
041_026.tif
041_027.tif
041_028.tif
041_029.tif
041_030.tif
041_031.tif
041_032.tif
041_033.tif
041_034.tif
041_035.tif
041_

053_008.tif
053_009.tif
053_010.tif
053_011.tif
053_012.tif
053_013.tif
053_014.tif
053_015.tif
053_016.tif
053_017.tif
053_018.tif
053_019.tif
053_020.tif
053_021.tif
053_022.tif
053_023.tif
053_024.tif
053_025.tif
053_026.tif
053_027.tif
053_028.tif
053_029.tif
053_030.tif
053_031.tif
053_032.tif
053_033.tif
053_034.tif
053_035.tif
053_036.tif
053_037.tif
053_038.tif
053_039.tif
053_040.tif
053_041.tif
053_042.tif
053_043.tif
053_044.tif
053_045.tif
053_046.tif
053_047.tif
053_048.tif
053_049.tif
053_050.tif
053_051.tif
053_052.tif
053_053.tif
053_054.tif
053_055.tif
053_056.tif
053_057.tif
053_058.tif
053_059.tif
053_060.tif
053_061.tif
053_062.tif
054_008.tif
054_009.tif
054_010.tif
054_011.tif
054_012.tif
054_013.tif
054_014.tif
054_015.tif
054_016.tif
054_017.tif
054_018.tif
054_019.tif
054_020.tif
054_021.tif
054_022.tif
054_023.tif
054_024.tif
054_025.tif
054_026.tif
054_027.tif
054_028.tif
054_029.tif
054_030.tif
054_031.tif
054_032.tif
054_033.tif
054_034.tif
054_035.tif
054_

065_011.tif
065_012.tif
065_013.tif
065_014.tif
065_015.tif
065_016.tif
065_017.tif
065_018.tif
065_019.tif
065_020.tif
065_021.tif
065_022.tif
065_023.tif
065_024.tif
065_025.tif
065_026.tif
065_027.tif
065_028.tif
065_029.tif
065_030.tif
065_031.tif
065_032.tif
065_033.tif
065_034.tif
065_035.tif
065_036.tif
065_037.tif
065_038.tif
065_039.tif
065_040.tif
065_041.tif
065_042.tif
065_043.tif
065_044.tif
065_045.tif
065_046.tif
065_047.tif
065_048.tif
065_049.tif
065_050.tif
065_051.tif
065_052.tif
065_053.tif
065_054.tif
065_055.tif
065_056.tif
065_057.tif
065_058.tif
065_059.tif
065_060.tif
065_061.tif
065_062.tif
065_063.tif
065_064.tif
065_065.tif
065_066.tif
065_067.tif
065_068.tif
066_012.tif
066_013.tif
066_014.tif
066_015.tif
066_016.tif
066_017.tif
066_018.tif
066_019.tif
066_020.tif
066_021.tif
066_022.tif
066_023.tif
066_024.tif
066_025.tif
066_026.tif
066_027.tif
066_028.tif
066_029.tif
066_030.tif
066_031.tif
066_032.tif
066_033.tif
066_034.tif
066_035.tif
066_036.tif
066_

...

089_024.tif
089_025.tif
089_026.tif
089_027.tif
089_028.tif
089_029.tif
089_030.tif
089_031.tif
089_032.tif
089_033.tif
089_034.tif
089_035.tif
089_036.tif
089_037.tif
089_038.tif
089_039.tif
089_040.tif
089_041.tif
089_042.tif
089_043.tif
089_044.tif
089_045.tif
089_046.tif
089_047.tif
089_048.tif
089_049.tif
089_050.tif
089_051.tif
089_052.tif
089_053.tif
089_054.tif
089_055.tif
089_056.tif
089_057.tif
089_058.tif
089_059.tif
089_060.tif
089_061.tif
089_062.tif
089_063.tif
089_064.tif
089_065.tif
089_066.tif
089_067.tif
089_068.tif
089_069.tif
089_070.tif
089_071.tif
089_072.tif
089_073.tif
089_074.tif
089_075.tif
089_076.tif
089_077.tif
089_078.tif
089_079.tif
089_080.tif
089_081.tif
089_082.tif
089_083.tif
089_084.tif
089_085.tif
089_086.tif
089_087.tif
089_088.tif
089_089.tif
089_090.tif
089_091.tif
089_092.tif
089_093.tif
089_094.tif
089_095.tif
089_096.tif
090_024.tif
090_025.tif
090_026.tif
090_027.tif
090_028.tif
090_029.tif
090_030.tif
090_031.tif
090_032.tif
090_033.tif
090_

098_032.tif
098_033.tif
098_034.tif
098_035.tif
098_036.tif
098_037.tif
098_038.tif
098_039.tif
098_040.tif
098_041.tif
098_042.tif
098_043.tif
098_044.tif
098_045.tif
098_046.tif
098_047.tif
098_048.tif
098_049.tif
098_050.tif
098_051.tif
098_052.tif
098_053.tif
098_054.tif
098_055.tif
098_056.tif
098_057.tif
098_058.tif
098_059.tif
098_060.tif
098_061.tif
098_062.tif
098_063.tif
098_064.tif
098_065.tif
098_066.tif
098_067.tif
098_068.tif
098_069.tif
098_070.tif
098_071.tif
098_072.tif
098_073.tif
098_074.tif
098_075.tif
098_076.tif
098_077.tif
098_078.tif
098_079.tif
098_080.tif
098_081.tif
098_082.tif
098_083.tif
098_084.tif
098_085.tif
098_086.tif
098_087.tif
098_088.tif
098_089.tif
098_090.tif
098_091.tif
098_092.tif
098_093.tif
098_094.tif
098_095.tif
098_096.tif
098_097.tif
098_098.tif
098_099.tif
098_100.tif
098_101.tif
098_102.tif
098_103.tif
098_104.tif
098_105.tif
098_106.tif
098_107.tif
098_108.tif
098_109.tif
098_110.tif
099_034.tif
099_035.tif
099_036.tif
099_037.tif
099_

107_050.tif
107_051.tif
107_052.tif
107_053.tif
107_054.tif
107_055.tif
107_056.tif
107_057.tif
107_058.tif
107_059.tif
107_060.tif
107_061.tif
107_062.tif
107_063.tif
107_064.tif
107_065.tif
107_066.tif
107_067.tif
107_068.tif
107_069.tif
107_070.tif
107_071.tif
107_072.tif
107_073.tif
107_074.tif
107_075.tif
107_076.tif
107_077.tif
107_078.tif
107_079.tif
107_080.tif
107_081.tif
107_082.tif
107_083.tif
107_084.tif
107_085.tif
107_086.tif
107_087.tif
107_088.tif
107_089.tif
107_090.tif
107_091.tif
107_092.tif
107_093.tif
107_094.tif
107_095.tif
107_096.tif
107_097.tif
107_098.tif
107_099.tif
107_100.tif
107_101.tif
107_102.tif
107_103.tif
107_104.tif
107_105.tif
107_106.tif
107_107.tif
107_108.tif
107_109.tif
107_110.tif
107_111.tif
107_112.tif
107_113.tif
107_114.tif
107_115.tif
107_116.tif
107_117.tif
107_118.tif
107_119.tif
107_120.tif
107_121.tif
107_122.tif
107_123.tif
107_124.tif
107_125.tif
108_053.tif
108_054.tif
108_055.tif
108_056.tif
108_057.tif
108_058.tif
108_059.tif
108_

115_132.tif
115_133.tif
115_134.tif
115_135.tif
115_136.tif
115_137.tif
115_138.tif
115_139.tif
115_140.tif
115_141.tif
115_142.tif
115_143.tif
115_144.tif
115_145.tif
115_146.tif
115_147.tif
115_148.tif
115_149.tif
115_150.tif
116_076.tif
116_077.tif
116_078.tif
116_079.tif
116_080.tif
116_081.tif
116_082.tif
116_083.tif
116_084.tif
116_085.tif
116_086.tif
116_087.tif
116_088.tif
116_089.tif
116_090.tif
116_091.tif
116_092.tif
116_093.tif
116_094.tif
116_095.tif
116_096.tif
116_097.tif
116_098.tif
116_099.tif
116_100.tif
116_101.tif
116_102.tif
116_103.tif
116_104.tif
116_105.tif
116_106.tif
116_107.tif
116_108.tif
116_109.tif
116_110.tif
116_111.tif
116_112.tif
116_113.tif
116_114.tif
116_115.tif
116_116.tif
116_117.tif
116_118.tif
116_119.tif
116_120.tif
116_121.tif
116_122.tif
116_123.tif
116_124.tif
116_125.tif
116_126.tif
116_127.tif
116_128.tif
116_129.tif
116_130.tif
116_131.tif
116_132.tif
116_133.tif
116_134.tif
116_135.tif
116_136.tif
116_137.tif
116_138.tif
116_139.tif
116_

129_126.tif
129_127.tif
129_128.tif
129_129.tif
129_130.tif
129_131.tif
129_132.tif
129_133.tif
129_134.tif
129_135.tif
129_136.tif
129_137.tif
129_138.tif
129_139.tif
129_140.tif
129_141.tif
129_142.tif
129_143.tif
129_144.tif
129_145.tif
129_146.tif
129_147.tif
129_148.tif
129_149.tif
129_150.tif
130_130.tif
130_131.tif
130_132.tif
130_133.tif
130_134.tif
130_135.tif
130_136.tif
130_137.tif
130_138.tif
130_139.tif
130_140.tif
130_141.tif
130_142.tif
130_143.tif
130_144.tif
130_145.tif
130_146.tif
130_147.tif
130_148.tif
130_149.tif
131_134.tif
131_135.tif
131_136.tif
131_137.tif
131_138.tif
131_139.tif
131_140.tif
131_141.tif
131_142.tif
131_143.tif
131_144.tif
131_145.tif
131_146.tif
131_147.tif
131_148.tif
101_081.tif
111_089.tif


In [8]:
len(Ratios)

729

In [9]:
print(num_files)
print(num_anno)
print(num_feature_file)

20502
7097
706


In [18]:
#non feature
Ratios = []
num_files = 0
num_anno = 0
num_feature_file = 0
num_nonfeature_file = 0
#select files with feature
# random_list = np.random.randint(2, size=81703)
random_list = np.concatenate((np.ones(706),np.zeros(6369 - 706))) #print(a,b,'\t',darray) 
np.random.shuffle(random_list)


# for non features
for mask in all_masks:
    num_files += 1
    # load annotations
    filename = os.path.join(ANNO_PATH, mask)
    tif = TIFF.open(filename)
    img = tif.read_image().astype(int) # 0 means annotated
    if 0 in np.unique(img):
        num_anno += 1
        
        #load mask image
        filename = os.path.join(MASK_PATH, mask)
        tif = TIFF.open(filename)
        mask_arr = tif.read_image().astype(int)
        mask_arr = np.where(mask_arr==2.0, 0.0, mask_arr)
        mask_arr = 1 - mask_arr # so 1 is the mask
        no_1 = np.count_nonzero(mask_arr)
        
        if no_1 == 0:
            num_nonfeature_file += 1 # count
            if random_list[num_nonfeature_file] != 1:
                continue
            filename = os.path.join(FRAME_PATH, mask)
            tif = TIFF.open(filename)
            frame_arr = tif.read_image()
            # save to file masks_1 and load frames
            np.save(os.path.join(SAVE_PATH+'frame_0/',mask[0:-4]+'.npy'), frame_arr)
            np.save(os.path.join(SAVE_PATH+'mask_0/',mask[0:-4]+'.npy'),mask_arr)              

In [19]:
print(num_files)
print(num_anno)
print(num_nonfeature_file)

20502
7097
6368


In [50]:
num_nonfeature_file

808

In [37]:
num_feature_file

1195

In [55]:
num_anno

53910

In [38]:
print(len(Ratios))

1251


In [23]:
array = np.nonzero(np.array(Ratios))
Ratios = np.array(Ratios)
print(Ratios[array])

[0.02612305 0.10205078 0.03735352 ... 0.10913086 0.01098633 0.02978516]


In [24]:
print(len(Ratios[array]))

1251


In [12]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter


In [13]:
n_bins = 100
plt.hist(Ratios[array], bins=n_bins, range = (0,0.30))

NameError: name 'array' is not defined

In [14]:
print(np.max(Ratios[array]))

NameError: name 'array' is not defined

In [43]:
np.random.randint(2, size=81703)

array([0, 1, 1, 1, 1, 0, 0, 0, 1, 1])